In [1]:
using Pkg
Pkg.activate("..")

 Activating environment at `~/Documents/Catlab/AlgebraicDynamics.jl/Project.toml`


In [32]:
using AlgebraicDynamics.CPortGraphDynam
using AlgebraicDynamics.DWDDynam
using AlgebraicDynamics.CPortGraphDynam: draw, barbell, gridpath, grid, meshpath
using Catlab.WiringDiagrams
using Catlab.WiringDiagrams.CPortGraphs
using Catlab.CategoricalAlgebra
using Catlab.Theories
using Catlab.Graphs
using Catlab

using DynamicalSystems
using PrettyTables

using Base.Iterators: repeated, flatten, take, cycle

In [3]:
# simulate a vector field using Euler's method

function simulate(f::ContinuousMachine{T}, nsteps::Int, h::Real, u0::Vector, xs=T[]) where T
    trajectory(euler_approx(f, h), u0, xs, nothing, nsteps)
end

function printsim(traj, stepfun, indxfun, shape)
    for u in stepfun(traj)
        pretty_table(reshape(indxfun(u), shape), equal_columns_width=true, noheader=true)
    end
end

printsim (generic function with 1 method)

In [4]:
# instantiate machines
α₁ = 1

fm = ContinuousMachine{Float64}(4, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 4))
fl = ContinuousMachine{Float64}(3, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 3))
fr = ContinuousMachine{Float64}(3, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 3))
ft = ContinuousMachine{Float64}(3, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 3))
fb = ContinuousMachine{Float64}(3, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 3))
fc = ContinuousMachine{Float64}(2, 1, (u,x,p,t) -> α₁ * (sum(x) .- u .* length(x)), u->repeated(u[1], 2))

ContinuousMachine(ℝ^1 × ℝ^2 → ℝ^1)

# Laplacians

In [9]:
d4 = @acset OpenCPortGraph begin
    Box = 3
    Port = 18
    Wire = 12
    OuterPort = 6
    box = Iterators.flatten([repeated(i, 6) for i in 1:3])
    con = [1,2,3,16,17,18]
    src = [4,5,6,7,8,9,10,11,12,13,14,15]
    tgt = [7,8,9,4,5,6,13,14,15,10,11,12]
end

gm = @acset OpenCPortGraph begin
    Box = 3
    Port = 10
    Wire = 4
    OuterPort = 6
    box = [1,1,1,2,2,2,2,3,3,3]
    src = [2, 4, 6, 8]
    tgt = [4, 2, 8, 6] 
    con = [3, 7, 10, 1, 5, 9]
end

Port,box
1,1
2,1
3,1
4,2
5,2
6,2
7,2
8,3
9,3
10,3


In [10]:
F = oapply(d4, oapply(gm, [ft,fm,fb]))
F2 = oapply(d4, [F, F, F])
F3 = oapply(d4, [F2,F2,F2])


ContinuousMachine(ℝ^81 × ℝ^6 → ℝ^81)

In [12]:
u₀ = zeros(81)
u₀[2] = 10

@time traj = simulate(F3, 50, 0.01, u₀, 10*ones(6))
@time traj = simulate(F3, 50, 0.01, u₀, 10*ones(6))

for u in traj
    pretty_table(reshape(u[1:27], (3,9)), equal_columns_width=true, noheader=true)
end

  5.619859 seconds (5.93 M allocations: 313.972 MiB, 2.57% gc time)
  0.049501 seconds (555.27 k allocations: 20.112 MiB)
┌──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┬──────┐
│  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │
│ 10.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │
│  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │
└──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┴──────┘
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ 0.2 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │
│ 9.7 │ 0.1 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │
│ 0.2 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │ 0.0 │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘
┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ 0.391 │ 0.003 │   0.0 │   0.0 │   0.0 │   0.0 │   0.0 │   0.0 │   0.0 │
│ 9.417 │ 0.193 │ 0.001 │   0.0 │   0.0 │   0.0 │   0.0 │   0.0 │   0.0 │
│ 0.391 │ 0.003 │   0.0 │   0.0 │   0.0 │   0.0 │  

# Advection-Diffusion

In [13]:
advecdiffuse(α₁, α₂) = begin
    diffop(u,p,t) = α₁ .* (sum(p) .- u .* length(p))
    advop(u,p,t)  = α₂ .* (p[end] .- u)
    ft = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 3))
    fm = ContinuousMachine{Float64}(4, 1, (u,p,q,t) -> diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 4))
    fb = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 3))
    return ft, fm, fb
end

advecdiffuse (generic function with 1 method)

In [14]:
ft, fm, fb = advecdiffuse(1.0,2.0)
F = oapply(d4, oapply(gm, [ft,fm,fb]))
traj = simulate(F, 48, 0.1, zeros(9), vcat(ones(3), zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,3))

┌──────────┬──────────┬──────────┐
│ 0.974792 │  0.89949 │ 0.674375 │
│ 0.974792 │  0.89949 │ 0.674375 │
│ 0.974792 │  0.89949 │ 0.674375 │
└──────────┴──────────┴──────────┘
┌──────────┬──────────┬──────────┐
│ 0.974824 │ 0.899569 │ 0.674472 │
│ 0.974824 │ 0.899569 │ 0.674472 │
│ 0.974824 │ 0.899569 │ 0.674472 │
└──────────┴──────────┴──────────┘
┌──────────┬──────────┬──────────┐
│ 0.974851 │ 0.899636 │ 0.674554 │
│ 0.974851 │ 0.899636 │ 0.674554 │
│ 0.974851 │ 0.899636 │ 0.674554 │
└──────────┴──────────┴──────────┘


In [15]:
F2 = oapply(d4, [F,F,F])
traj = simulate(F2, 16, 0.1, zeros(27), vcat(ones(3), zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────────
│   0.930837 │   0.801814 │   0.621902 │   0.424257 │   0.249651 │   0.124606  ⋯
│   0.930837 │   0.801814 │   0.621902 │   0.424257 │   0.249651 │   0.124606  ⋯
│   0.930837 │   0.801814 │   0.621902 │   0.424257 │   0.249651 │   0.124606  ⋯
└────────────┴────────────┴────────────┴────────────┴────────────┴──────────────
                                                               3 columns omitted
┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────────
│   0.938684 │    0.82253 │   0.656111 │    0.46609 │   0.289528 │   0.154851  ⋯
│   0.938684 │    0.82253 │   0.656111 │    0.46609 │   0.289528 │   0.154851  ⋯
│   0.938684 │    0.82253 │   0.656111 │    0.46609 │   0.289528 │   0.154851  ⋯
└────────────┴────────────┴────────────┴────────────┴────────────┴──────────────
                                                               3 columns omitted
┌───────────┬───────────┬───

In [16]:
ft, fm, fb = advecdiffuse(1.0,4.0)
F = oapply(d4, oapply(gm, [ft,fm,fb]))
F2 = oapply(d4, [F,F,F])
traj = simulate(F2, 16, 0.1, zeros(27), vcat(ones(3), zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.992416 │ 0.970882 │ 0.924214 │ 0.841355 │ 0.717722 │ 0.561297 │ 0.393356 │ ⋯
│ 0.992416 │ 0.970882 │ 0.924214 │ 0.841355 │ 0.717722 │ 0.561297 │ 0.393356 │ ⋯
│ 0.992416 │ 0.970882 │ 0.924214 │ 0.841355 │ 0.717722 │ 0.561297 │ 0.393356 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.994055 │ 0.976983 │ 0.939262 │ 0.870421 │ 0.763896 │ 0.622715 │ 0.462088 │ ⋯
│ 0.994055 │ 0.976983 │ 0.939262 │ 0.870421 │ 0.763896 │ 0.622715 │ 0.462088 │ ⋯
│ 0.994055 │ 0.976983 │ 0.939262 │ 0.870421 │ 0.763896 │ 0.622715 │ 0.462088 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬─────

In [17]:
traj = simulate(F2, 16, 0.1, zeros(27), vcat([0,1,0], zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────
│  0.132515 │  0.205674 │  0.237915 │  0.238742 │  0.214474 │  0.172443 │  0.1 ⋯
│  0.727386 │  0.559535 │  0.448385 │  0.363872 │  0.288773 │   0.21641 │  0.1 ⋯
│  0.132515 │  0.205674 │  0.237915 │  0.238742 │  0.214474 │  0.172443 │  0.1 ⋯
└───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────
                                                               3 columns omitted
┌───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────
│  0.133061 │  0.207705 │  0.242924 │  0.248415 │  0.229835 │  0.192863 │  0.1 ⋯
│  0.727934 │  0.561573 │  0.453415 │  0.373592 │  0.304227 │   0.23699 │  0.1 ⋯
│  0.133061 │  0.207705 │  0.242924 │  0.248415 │  0.229835 │  0.192863 │  0.1 ⋯
└───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────
                                                               3 columns omitted
┌───────────┬───────────┬───

# Reaction Diffusion Advection

In [18]:
RDA(α₀, α₁, α₂) = begin
    diffop(u,p,t) = α₁ .* (sum(p) .- u .* length(p))
    advop(u,p,t)  = α₂ .* (p[end] .- u)
    ft = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 3))
    fm = ContinuousMachine{Float64}(4, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 4))
    fb = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p,t), u->repeated(u[1], 3))
    return ft, fm, fb
end

RDA (generic function with 1 method)

In [19]:
ft, fm, fb = RDA(0.1, 1.0,2.0)
F = oapply(d4, oapply(gm, [ft,fm,fb]))
traj = simulate(F, 48, 0.1, zeros(9), vcat(ones(3), zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,3))

┌──────────┬──────────┬──────────┐
│  1.01941 │ 0.976261 │ 0.750543 │
│  1.01941 │ 0.976261 │ 0.750543 │
│  1.01941 │ 0.976261 │ 0.750543 │
└──────────┴──────────┴──────────┘
┌──────────┬──────────┬──────────┐
│  1.01947 │ 0.976397 │  0.75071 │
│  1.01947 │ 0.976397 │  0.75071 │
│  1.01947 │ 0.976397 │  0.75071 │
└──────────┴──────────┴──────────┘
┌──────────┬──────────┬──────────┐
│  1.01951 │ 0.976514 │ 0.750852 │
│  1.01951 │ 0.976514 │ 0.750852 │
│  1.01951 │ 0.976514 │ 0.750852 │
└──────────┴──────────┴──────────┘


In [20]:
F2 = oapply(d4, [F,F,F])
traj = simulate(F2, 16, 0.1, zeros(27), vcat(ones(3), zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────────
│   0.963798 │   0.851088 │    0.67071 │   0.461517 │   0.272379 │   0.135763  ⋯
│   0.963798 │   0.851088 │    0.67071 │   0.461517 │   0.272379 │   0.135763  ⋯
│   0.963798 │   0.851088 │    0.67071 │   0.461517 │   0.272379 │   0.135763  ⋯
└────────────┴────────────┴────────────┴────────────┴────────────┴──────────────
                                                               3 columns omitted
┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────────
│   0.973026 │   0.875374 │   0.710611 │   0.509976 │   0.318183 │    0.17016  ⋯
│   0.973026 │   0.875374 │   0.710611 │   0.509976 │   0.318183 │    0.17016  ⋯
│   0.973026 │   0.875374 │   0.710611 │   0.509976 │   0.318183 │    0.17016  ⋯
└────────────┴────────────┴────────────┴────────────┴────────────┴──────────────
                                                               3 columns omitted
┌───────────┬───────────┬───

In [21]:
traj = simulate(F2, 64, 0.1, zeros(27), vcat([0,1,0], zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.195228 │ 0.295545 │ 0.350964 │  0.38411 │ 0.404483 │ 0.414679 │  0.41179 │ ⋯
│ 0.661523 │ 0.512975 │ 0.452351 │ 0.431386 │ 0.426527 │ 0.424958 │ 0.416582 │ ⋯
│ 0.195228 │ 0.295545 │ 0.350964 │  0.38411 │ 0.404483 │ 0.414679 │  0.41179 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.195273 │  0.29569 │ 0.351302 │ 0.384773 │  0.40564 │ 0.416506 │ 0.414377 │ ⋯
│ 0.661567 │ 0.513121 │ 0.452688 │ 0.432048 │ 0.427684 │ 0.426785 │ 0.419168 │ ⋯
│ 0.195273 │  0.29569 │ 0.351302 │ 0.384773 │  0.40564 │ 0.416506 │ 0.414377 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬─────

In [22]:
ft, fm, fb = RDA(-0.4, 1.0,4.0)
F = oapply(d4, oapply(gm, [ft,fm,fb]))
F2 = oapply(d4, [F,F,F])
traj = simulate(F2, 128, 0.1, zeros(27), vcat([0,1,0], zeros(3)))
printsim(traj, t->t[end-2:end], identity, (3,9))

┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.114967 │ 0.169793 │ 0.191493 │ 0.195271 │ 0.189648 │ 0.179307 │  0.16645 │ ⋯
│ 0.680962 │ 0.490143 │ 0.372809 │ 0.297895 │ 0.247733 │ 0.212182 │ 0.185053 │ ⋯
│ 0.114967 │ 0.169793 │ 0.191493 │ 0.195271 │ 0.189648 │ 0.179307 │  0.16645 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──
│ 0.114967 │ 0.169793 │ 0.191493 │ 0.195271 │ 0.189648 │ 0.179307 │  0.16645 │ ⋯
│ 0.680962 │ 0.490143 │ 0.372809 │ 0.297895 │ 0.247733 │ 0.212182 │ 0.185053 │ ⋯
│ 0.114967 │ 0.169793 │ 0.191493 │ 0.195271 │ 0.189648 │ 0.179307 │  0.16645 │ ⋯
└──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──
                                                               2 columns omitted
┌──────────┬──────────┬─────

# Taller Pipe

In [23]:
RDA(α₀, α₁, α₂) = begin
    diffop(u,p,t) = α₁ .* (sum(p) .- u .* length(p))
    advop(u,p,t)  = begin  α₂ .* (p .- u) end 
    ft = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 3))
    fm = ContinuousMachine{Float64}(4, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 4))
    fb = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 3))
    return ft, fm, fb
end

function executerda(size, parameters)
    function gensim(n::Int, depth::Int)
        ft,fm,fb = RDA(parameters...)
        F = oapply(apex(meshpath(n)), vcat([ft], collect(repeated(fm, n-2)), [fb]))
        l = 2^(depth-1)
        oapply(apex(gridpath(l, n)), collect(repeated(F, l)))
    end

    function runsim(F, n, depth)
        inputs = zeros(n)
        inputs[1] = 1
        inputs[2] = 1
        inputs[end] = 1
        # inputs[floor(Int, n/2)] = 1 
        l = 2^(depth-1)
        traj = simulate(F, 12, 0.1, zeros(l*n), vcat(inputs, zeros(n)))
        printsim(traj, t->t[end:end], u->u, (n,l))
        return traj
    end
    runsim(gensim(size...), size...)
end

executerda (generic function with 1 method)

In [24]:
executerda((5,4), [0.0,0.0,4.0]);
executerda((5,4), [0.0,0.1,4.0]);
executerda((5,4), [0.4,1.0,4.0]);
executerda((5,4), [0.4,1.0,6.0]);
executerda((6,5), [0.4,1.0,6.0]);

┌───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────
│   1.97823 │   2.81352 │   3.24054 │   3.02195 │   2.23578 │   1.28407 │  0.5 ⋯
│  0.997823 │  0.980409 │  0.916557 │  0.774663 │  0.561822 │  0.334791 │  0.1 ⋯
│       0.0 │       0.0 │       0.0 │       0.0 │       0.0 │       0.0 │      ⋯
│       0.0 │       0.0 │       0.0 │       0.0 │       0.0 │       0.0 │      ⋯
│  0.997823 │  0.980409 │  0.916557 │  0.774663 │  0.561822 │  0.334791 │  0.1 ⋯
└───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────
                                                               2 columns omitted
┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────────
│    1.94264 │     2.7256 │    3.11976 │    2.92159 │    2.19586 │    1.29534  ⋯
│   0.995199 │   0.988137 │   0.936244 │   0.800149 │   0.585901 │   0.353128  ⋯
│  0.0236432 │  0.0437006 │  0.0551726 │  0.0536804 │  0.0406897 │  0.0237372  ⋯
│  0.0232058 │  0.0424572 │ 

In [25]:
@time executerda((32,5), [0.4,1.0,6.0]);
@time executerda((32,5), [0.4,1.0,6.0]);

┌─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────
│     2.05131 │     3.14969 │      4.2456 │     5.13089 │     5.58131 │     5. ⋯
│     1.08797 │     1.29436 │     1.55775 │     1.80151 │     1.89619 │      1 ⋯
│    0.143333 │    0.280995 │    0.413096 │    0.516938 │    0.575735 │    0.5 ⋯
│   0.0201835 │   0.0523775 │   0.0895168 │     0.12469 │    0.142001 │    0.1 ⋯
│  0.00294406 │  0.00884554 │   0.0171198 │   0.0246706 │   0.0303961 │   0.02 ⋯
│ 0.000414385 │  0.00139449 │  0.00275783 │  0.00433829 │  0.00483453 │  0.004 ⋯
│  5.62943e-5 │ 0.000186493 │ 0.000406965 │ 0.000565392 │ 0.000715353 │ 0.0004 ⋯
│  6.30038e-6 │  2.35545e-5 │  4.35996e-5 │  7.11613e-5 │  5.63707e-5 │  5.435 ⋯
│  6.91148e-7 │    2.055e-6 │  4.73848e-6 │  4.54509e-6 │  5.54631e-6 │        ⋯
│  4.74681e-8 │   1.9503e-7 │  2.41129e-7 │  3.96165e-7 │         0.0 │        ⋯
│   3.9634e-9 │    7.595e-9 │   1.8865e-8 │         0.0 │         0.0 │        ⋯
│   1.078e-10 │    5.39e-10 

# RDA Benchmark

In [26]:
RDA(α₀, α₁, α₂) = begin
    diffop(u,p,t) = α₁ .* (sum(p) .- u .* length(p))
    advop(u,p,t)  = begin  α₂ .* (p .- u) end 
    ft = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 3))
    fm = ContinuousMachine{Float64}(4, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 4))
    fb = ContinuousMachine{Float64}(3, 1, (u,p,q,t) -> α₀ .* u .+ diffop(u,p,u) .+ advop(u,p[2],t), u->repeated(u[1], 3))
    return ft, fm, fb
end

RDA (generic function with 1 method)

In [50]:
function executerda(size, parameters, nsteps=10, stepsize=0.1)
    n, depth = size
    ft,fm,fb = RDA(parameters...)
    F = oapply(apex(meshpath(n)), vcat([ft], collect(repeated(fm, n-2)), [fb]))
    l = 2^(depth-1)
    F₁ = oapply(apex(gridpath(l, n)), collect(repeated(F, l)))
    inputs = ones(n)
    @time traj = simulate(F₁, nsteps, stepsize, zeros(l*n), vcat(inputs, zeros(n)))
    # printsim(traj, t->t[end:end], u->u, (n,l))
end

function executerda_ocomposed(size, parameters, nsteps=10, stepsize=0.1)
    n, depth = size
    l = 2^(depth-1)
    ft,fm,fb = RDA(parameters...)
    G = grid(l, n)
    F = oapply(G, take(cycle(flatten(([ft], repeated(fm, n-2), [fb]))), nparts(G, :Box))|> collect)
    inputs = ones(n)
    @time traj = simulate(F, nsteps, stepsize, zeros(l*n), vcat(inputs, zeros(n)))
    # printsim(traj, t->t[end:end], u->u, (n,l))
end

executerda_ocomposed (generic function with 3 methods)

In [57]:
depth = 5
println("Benchmark Nested")
executerda((64,depth), [0.4,1.0,4.0], 10, 0.1)
executerda((64,depth), [0.4,1.0,4.0], 10, 0.1)
executerda((64,depth), [0.4,1.0,4.0], 10, 0.1)

Benchmark Nested
  1.965224 seconds (1.54 M allocations: 57.215 MiB)
  0.148078 seconds (959.85 k allocations: 33.129 MiB, 23.81% gc time)
  0.075061 seconds (959.85 k allocations: 33.129 MiB)


1024-dimensional Dataset{Float64} with 11 points
 0.0      0.0       0.0       0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5      0.5       0.5       0.5          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.92     0.72      0.72      0.72         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.1978   0.8618    0.8418    0.8418       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.40515  0.957792  0.919392  0.917392     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.56407  1.02934   0.972192  0.967872  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.69115  1.08469   1.01039   1.00321      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.79512  1.12915   1.03909   1.02891      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.88203  1.16572   1.0614    1.04812      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.95583  1.19641   1.07918   1.06284      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.01931  1.22255   1.09366   1.07433   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [58]:
println("Benchmark Flattened")
executerda_ocomposed((64,depth), [0.4,1.0,4.0], 10, 0.1)
executerda_ocomposed((64,depth), [0.4,1.0,4.0], 10, 0.1)
executerda_ocomposed((64,depth), [0.4,1.0,4.0], 10, 0.1)

Benchmark Flattened
  0.047689 seconds (687.88 k allocations: 23.366 MiB)
  0.080779 seconds (687.88 k allocations: 23.366 MiB, 30.91% gc time)
  0.049719 seconds (687.88 k allocations: 23.366 MiB)


1024-dimensional Dataset{Float64} with 11 points
 0.0      0.0       0.0       0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5      0.5       0.5       0.5          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.92     0.72      0.72      0.72         0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.1978   0.8618    0.8418    0.8418       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.40515  0.957792  0.919392  0.917392     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.56407  1.02934   0.972192  0.967872  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.69115  1.08469   1.01039   1.00321      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.79512  1.12915   1.03909   1.02891      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.88203  1.16572   1.0614    1.04812      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.95583  1.19641   1.07918   1.06284      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.01931  1.22255   1.09366   1.07433   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0